In [29]:
import os

old_folder = r'D:\Cosmos_HighRes\20241130_CTP11_SABER_test_HCl\Alignment'

before_position_file = os.path.join(old_folder, '10x_positions_before.txt')

new_folder = r'D:\Cosmos_HighRes\20241130_CTP11_SABER_test_HCl\Alignment'

after_position_file = os.path.join(new_folder, '10x_positions_after.txt')

os.path.isfile(before_position_file), os.path.isfile(after_position_file)



(True, True)

# Calculate rotation matrix

In [30]:
import numpy as np
import os
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [31]:
R, T = align_manual_points(before_position_file, after_position_file, save=False, save_folder=new_folder)
R, T

(array([[ 0.99994334, -0.01064494],
        [ 0.01064494,  0.99994334]]),
 array([297.41474728, -51.68798819]))

In [32]:
np.arccos(R[0,1])

1.581441465620694

# Translate 60x positions

In [33]:
old_positions = np.loadtxt(os.path.join(os.path.dirname(old_folder), 'positions_all.txt'), delimiter=',')

In [34]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[ 607.21287145 1815.92004677]
 [ 609.39508369 2020.90843168]
 [ 611.57729593 2225.8968166 ]
 [ 613.75950818 2430.88520151]
 [ 615.94172042 2635.87358643]
 [ 618.12393267 2840.86197134]
 [ 825.29452983 3043.66814401]
 [ 823.11231758 2838.6797591 ]
 [ 820.93010534 2633.69137418]
 [ 818.74789309 2428.70298927]
 [ 816.56568085 2223.71460435]
 [ 814.3834686  2018.72621944]
 [ 812.20125636 1813.73783453]
 [1017.18964127 1811.55562228]
 [1019.37185352 2016.5440072 ]
 [1021.55406576 2221.53239211]
 [1023.73627801 2426.52077702]
 [1025.91849025 2631.50916194]
 [1028.1007025  2836.49754685]
 [1030.28291474 3041.48593177]
 [1235.27129965 3039.30371952]
 [1233.08908741 2834.31533461]
 [1230.90687517 2629.32694969]
 [1228.72466292 2424.33856478]
 [1226.54245068 2219.35017987]
 [1224.36023843 2014.36179495]
 [1222.17802619 1809.37341004]
 [1427.1664111  1807.19119779]
 [1429.34862335 2012.17958271]
 [1431.53083559 2217.16796762]
 [1433.71304784 2422.15635254]
 [1435.89526008 2627.14473745]
 [1438.0

In [35]:
save_filename = os.path.join(new_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Cosmos_HighRes\20241130_CTP11_SABER_test_HCl\Alignment\translated_positions_all.txt


# Manual adjust positions

In [36]:
manual_positions_before_file = os.path.join(os.path.dirname(old_folder), '60x_positions_before.txt')
print(manual_positions_before_file)
manual_positions = np.loadtxt(manual_positions_before_file, delimiter=',')
print(manual_positions)

D:\Cosmos_HighRes\20241130_CTP11_SABER_test_HCl\60x_positions_before.txt
[[-315.78 1347.73]
 [1927.18 3611.42]
 [1631.78  813.02]]


In [37]:
translated_manual_positions = np.dot(manual_positions, R) + T
print('translated manual positions:')
print(np.round(translated_manual_positions, 1))

translated manual positions:
[[  -4.  1299.3]
 [2262.9 3539. ]
 [1937.8  743.9]]


In [38]:
manual_real_positions = [
    [8.5, 1302],
    [2276.0, 3538],
    [1954, 745],
    #[5545.0, -763],
]
manual_shifts = np.array(manual_real_positions) - translated_manual_positions[:len(manual_real_positions)]
manual_shifts

array([[12.50085896,  2.67289072],
       [13.07110353, -1.01268134],
       [16.24316036,  1.08424962]])

In [39]:
manual_shift = np.mean(manual_shifts, axis=0)
manual_shift

array([13.93837428,  0.91481967])

In [40]:
adjusted_new_positions = new_positions + manual_shift

In [41]:
adjusted_new_positions[0], old_positions[0]

(array([ 621.15124573, 1816.83486644]), array([ 289.9, 1870.8]))

In [42]:
adj_save_filename = os.path.join(new_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Cosmos_HighRes\20241130_CTP11_SABER_test_HCl\Alignment\adjusted_translated_positions_all.txt
